# Введение в глубинное обучение, ФКН ВШЭ

## Домашнее задание 3. Обработка текстов.

### Общая информация

Дата выдачи: 13.01.2022

Мягкий дедлайн: 23:59MSK 6.02.2022

Жесткий дедлайн: 23:59MSK 10.02.2022

Оценка после штрафа после мягкого дедлайна вычисляется по формуле $M_{penalty} = M_{full} \cdot 0.85^{t/1440}$, где $M_{full}$ — полная оценка за работу без учета штрафа, а $t$ — время в минутах, прошедшее после мягкого дедлайна (округление до двух цифр после запятой). Таким образом, спустя первые сутки после мягкого дедлайна вы не можете получить оценку выше 8.5, а если сдать перед самым жестким дедлайном, то ваш максимум — 5.22 балла.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В данном домашнем задании вам предстоит предсказывать пользовательскую оценку отеля по тексту отзыва. Нужно обучиться на данных с кэггла и заслать в [соревнование](https://www.kaggle.com/t/3e8fa6cec6d048bf8e93fb72e441d88c) предикт. По той же ссылке можете скачать данные.

Мы собрали для вас отзывы по 1500 отелям из совершенно разных уголков мира. Что это за отели - секрет. Вам дан текст отзыва и пользовательская оценка отеля. Ваша задача - научиться предсказывать оценку отеля по отзыву.

Главная метрика - Mean Absolute Error (MAE). Во всех частях домашней работы вам нужно получить значение MAE не превышающее 0.92 на публичном лидерборде. В противном случае мы будем вынуждены не засчитать задание :( 

#### Про данные:
Каждое ревью состоит из двух текстов: positive и negative - плюсы и минусы отеля. В столбце score находится оценка пользователя - вещественное число 0 до 10. Вам нужно извлечь признаки из этих текстов и предсказать по ним оценку.

Для локального тестирования используйте предоставленное разбиение на трейн и тест.

Good luck & have fun! 💪

#### Использовать любые данные для обучения кроме предоставленных организаторами строго запрещено. В последней части можно использовать предобученные модели из библиотеки `transformers`.

In [ ]:
PATH_TO_TRAIN_DATA = 'train.csv'
PATH_TO_TEST_DATA = 'test.csv'

In [ ]:
import pandas as pd

df = pd.read_csv(PATH_TO_TRAIN_DATA)

In [ ]:
df_kaggle = pd.read_csv(PATH_TO_TEST_DATA)

Предобработка текста может сказываться на качестве вашей модели.
Сделаем небольшой препроцессинг текстов: удалим знаки препинания, приведем все слова к нижнему регистру. 
Однако можно не ограничиваться этим набором преобразований. Подумайте, что еще можно сделать с текстами, чтобы помочь будущим моделям? Добавьте преобразования, которые могли бы помочь по вашему мнению.

Также мы добавили разбиение текстов на токены. Теперь каждая строка-ревью стала массивом токенов.

In [ ]:
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, SnowballStemmer 

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
#lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

def preprocess(text): #с семинара
    text = list(filter(str.isalpha, word_tokenize(text.lower())))
    #text = list(lemmatizer.lemmatize(word) for word in text)
    text = list(stemmer.stem(word) for word in text)
    text = list(word for word in text if word not in stop_words)
    return ' '.join(text)

def process_text(text):
    return [word for word in word_tokenize(text.lower()) if word not in string.punctuation]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['negative'] = df['negative'].apply(preprocess)
df['positive'] = df['positive'].apply(preprocess)

df.head()

,review_id,negative,positive,score
0,00003c6036f30f590c0ac435efb8739b,"[issu, wifi, connect]",[posit],7.1
1,00004d18f186bf2489590dc415876f73,"[tv, work]",[posit],7.5
2,0000cf900cbb8667fad33a717e9b1cf4,[pillow],"[beauti, room, great, locat, love, staff]",10.0
3,0000df16edf19e7ad9dd8c5cd6f6925e,"[veri, busi]",[locat],5.4
4,00025e1aa3ac32edb496db49e76bbd00,"[room, could, bit, refurbish, could, corridor,...","[nice, breakfast, handi, victoria, train, stat...",6.7


In [ ]:
df_kaggle['negative'] = df_kaggle['negative'].apply(preprocess)
df_kaggle['positive'] = df_kaggle['positive'].apply(preprocess)

df_kaggle.head()

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, random_state=1412) # <- для локального тестирования

### Часть 1. 1 балл

Обучите логистическую регрессию на TF-IDF векторах текстов.

Код частично взят [отсюда](https://datascience.stackexchange.com/questions/18581/same-tf-idf-vectorizer-for-2-data-inputs)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error
from scipy.sparse import hstack

In [ ]:
#подготовка train и test
train_neg = df_train.negative
train_pos = df_train.positive

y_train = df_train.score

test_neg = df_test.negative
test_pos = df_test.positive

y_test = df_test.score

kaggle_neg = df_kaggle.negative
kaggle_pos = df_kaggle.positive

In [ ]:
#векторизация
text_transformer = TfidfVectorizer()

train_neg_vec = text_transformer.fit_transform(train_neg)
test_neg_vec = text_transformer.transform(test_neg)
kaggle_neg_vec = text_transformer.transform(kaggle_neg)

text_transformer = TfidfVectorizer()

train_pos_vec = text_transformer.fit_transform(train_pos)
test_pos_vec = text_transformer.transform(test_pos)
kaggle_pos_vec = text_transformer.transform(kaggle_pos)

In [ ]:
#объединение
x_train = hstack((train_neg_vec, train_pos_vec))
x_test = hstack((test_neg_vec, test_pos_vec))
x_kaggle = hstack((kaggle_neg_vec, kaggle_pos_vec))

In [ ]:
lr = Ridge()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

In [ ]:
print("MAE =", mean_absolute_error(y_test, y_pred))

MAE =  0.839509092705269


MAE для разных тестов:

LinReg, PorterStemmer : 0.9809863947284235

LinReg, PorterStemmer : 0.9771248627996291

LinReg, WordNetLemmatizer : 1.0577106356614245

Ridge, SnowballStemmer : 0.839509092705269

Предскажите этой моделью тестовые данные из [соревнования](https://www.kaggle.com/t/3e8fa6cec6d048bf8e93fb72e441d88c) и сделайте сабмит. Какой у вас получился скор? Прикрепите скриншот из кэггла.

In [ ]:
#для kaggle
y_pred = lr.predict(x_kaggle)

res = pd.DataFrame()
res['review_id'] = df_kaggle.review_id
res['score'] = y_pred.tolist()
res.head()

,review_id,score
0,00026f564b258ad5159aab07c357c4ca,4.543266
1,000278c73da08f4fcb857fcfe4ac6417,10.063655
2,000404f843e756fe3b2a477dbefa5bd4,6.448572
3,000a66d32bcf305148d789ac156dd512,6.633642
4,000bf1d8c5110701f459ffbedbf0d546,9.222415


In [ ]:
res.to_csv(r'prediction.csv', header=['review_id', 'score'], index=None, sep=',', mode='a')

![](https://drive.google.com/uc?export=view&id=1SEsDqwLlB6QIeX-k4NxvlA76sdyWslqo)

### Часть 2. 2 балла

Обучите логистическую регрессию на усредненных Word2Vec векторах. 

Для этого задания я брала код [отсюда](https://towardsdatascience.com/using-word2vec-to-analyze-news-headlines-and-predict-article-success-cdeda5f14751) и [отсюда](https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial)

In [ ]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge

In [ ]:
df = pd.read_csv(PATH_TO_TRAIN_DATA)

In [ ]:
def preprocess2(text):
    text = list(filter(str.isalpha, word_tokenize(text.lower())))
    text = list(word for word in text if word not in stop_words)
    return text

In [ ]:
df['negative'] = df['negative'].apply(preprocess2)
df['positive'] = df['positive'].apply(preprocess2)

df.head()

,review_id,negative,positive,score
0,00003c6036f30f590c0ac435efb8739b,"[issues, wifi, connection]",[positive],7.1
1,00004d18f186bf2489590dc415876f73,"[tv, working]",[positive],7.5
2,0000cf900cbb8667fad33a717e9b1cf4,[pillows],"[beautiful, room, great, location, lovely, staff]",10.0
3,0000df16edf19e7ad9dd8c5cd6f6925e,[business],[location],5.4
4,00025e1aa3ac32edb496db49e76bbd00,"[rooms, could, bit, refurbishment, could, corr...","[nice, breakfast, handy, victoria, train, stat...",6.7


In [ ]:
#создаем список всех слов датафрейма
tokenized_corpus = []
for index, row in df.iterrows():
  tokenized_corpus.append(row['negative'])
  tokenized_corpus.append(row['positive'])

vocabulary = []

for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

In [ ]:
#word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
#idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

#vocabulary_size = len(vocabulary)

In [ ]:
model = Word2Vec(min_count=20, window=2, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20)

model.build_vocab(tokenized_corpus, progress_per=10000)

model.train(tokenized_corpus, total_examples=model.corpus_count, epochs=30, report_delay=1)



(20677446, 55530480)

In [ ]:
words_filtered = [word for word in vocabulary if word in model.wv.vocab]

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, random_state=1412) # <- для локального тестирования

In [ ]:
train_arrays_list = []
for index, row in df_train.iterrows():
  review = row.negative.copy() + row.positive.copy()
  review = [word for word in review if word in model.wv.vocab]
  if len(review) >= 1:
      train_arrays_list.append(np.mean(model[review], axis=0))
  else:
      train_arrays_list.append([])
  #tokenized_corpus.append(row['negative'])
  #tokenized_corpus.append(row['positive'])


test_arrays_list = []
for index, row in df_test.iterrows():
  review = row.negative.copy() + row.positive.copy()
  review = [word for word in review if word in model.wv.vocab]
  if len(review) >= 1:
      test_arrays_list.append(np.mean(model[review], axis=0))
  else:
      test_arrays_list.append([])
  #tokenized_corpus.append(row['negative'])
  #tokenized_corpus.append(row['positive'])


X_train = np.array(train_arrays_list)
X_test = np.array(test_arrays_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

In [ ]:
lr = Ridge()
lr.fit(X_train, df_train.score)

y_pred = lr.predict(X_test)

TypeError: ignored

In [ ]:
print("MAE =", mean_absolute_error(df_test.score, y_pred))

Усредняя w2v вектора, мы предполагаем, что каждое слово имеет равноценный вклад в смысл предложения, однако это может быть не совсем так. Теперь попробуйте воспользоваться другой концепцией и перевзвесить слова при получении итогового эмбеддинга текста. В качестве весов используйте IDF (Inverse document frequency)

In [ ]:
def calc_idf(texts):
    pass

Проведите эксперименты с размерностью эмбеддинга. Для каждого из двух методов постройте график зависимости качества модели от размерности эмбеддинга. 

#### Сделайте выводы:

Теперь попробуйте обучить логистическую регрессию на любых других эмбеддингах размерности 300 и сравните качество с Word2Vec.
#### Выводы:
`<ВАШ ТЕКСТ ЗДЕСЬ>`

Предскажите вашей лучшей моделью из этого задания тестовые данные из [соревнования](https://www.kaggle.com/t/3e8fa6cec6d048bf8e93fb72e441d88c) и сделайте сабмит. Какой у вас получился скор? Прикрепите скриншот из кэггла.

### Часть 3. 4 балла

Теперь давайте воспользуемся более продвинутыми методами обработки текстовых данных, которые мы проходили в нашем курсе. Обучите RNN/Transformer для предсказания пользовательской оценки. Получите ошибку меньше, чем во всех вышеперечисленных методах.

Если будете обучать RNN, попробуйте ограничить максимальную длину предложения. Некоторые отзывы могут быть слишком длинные относительно остальных.

Чтобы пользоваться DataLoader, все его элементы должны быть одинаковой размерности. Для этого вы можете добавить нулевой паддинг ко всем предложениям (см пример pad_sequence)

Код взят [отсюда](https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f)

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
df = pd.read_csv(PATH_TO_TRAIN_DATA)

In [ ]:
df["text"] = df["negative"] + ' ' + df["positive"]

In [ ]:
df.head()

,review_id,negative,positive,score,text
0,00003c6036f30f590c0ac435efb8739b,There were issues with the wifi connection,No Positive,7.1,There were issues with the wifi connection No...
1,00004d18f186bf2489590dc415876f73,TV not working,No Positive,7.5,TV not working No Positive
2,0000cf900cbb8667fad33a717e9b1cf4,More pillows,Beautiful room Great location Lovely staff,10.0,More pillows Beautiful room Great location L...
3,0000df16edf19e7ad9dd8c5cd6f6925e,Very business,Location,5.4,Very business Location
4,00025e1aa3ac32edb496db49e76bbd00,Rooms could do with a bit of a refurbishment ...,Nice breakfast handy for Victoria train stati...,6.7,Rooms could do with a bit of a refurbishment ...


In [ ]:
WORDS = set()
for sent in list(df['positive']):
    for w in sent:
        WORDS.add(w)
        
for sent in list(df['negative']):
    for w in sent:
        WORDS.add(w)

In [ ]:
int2word = dict(enumerate(tuple(WORDS)))
word2int = {w: ii for ii, w in int2word.items()}

In [ ]:
MAX_LEN = max(max(df['positive'].apply(len)), max(df['negative'].apply(len)))

In [ ]:
MAX_LEN

1965

In [ ]:
from torch.nn.utils.rnn import pad_sequence

train_pos_pad = pad_sequence([torch.as_tensor([word2int[w] for w in seq][:MAX_LEN]) for seq in df_train['positive']], 
                           batch_first=True)

In [ ]:
train_pos_pad = pad_sequence([torch.as_tensor([word2int[w] for w in seq][:MAX_LEN]) for seq in df_train['negative']], 
                           batch_first=True)

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [label for label in df['score']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

80000 10000 10000


In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

In [ ]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        ## TODO
        pass
        
    def __len__(self):
        ## TODO
        pass
    
    def __getitem__(self, idx):
        ## TODO
        pass

In [ ]:
BATCH_SIZE = 1

train_dataset = ReviewsDataset(df_train)
test_dataset = ReviewsDataset(df_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
NUM_EPOCHS = 1

for n in range(NUM_EPOCHS):
    model.train()
    ## TODO

NameError: ignored

### Контест (до 3 баллов)

По итогам всех ваших экспериментов выберите модель, которую считаете лучшей. Сделайте сабмит в контест. В зависимости от вашего скора на публичном лидерборде, мы начислим вам баллы:

 - <0.76 - 3 балла
 - [0.76; 0.78) - 2 балла
 - [0.78; 0.8) - 1 балл